In [3]:
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
def get_similar_users(user_id, user_similarity_df, top_n=5):
    """Find top-N similar users"""
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:top_n+1]
    return similar_users
def recommend_movies(user_id, user_similarity_df, user_item_matrix, top_n=5):
    """Recommend top-N movies for a user based on similar users"""
    similar_users = get_similar_users(user_id, user_similarity_df)
    similar_users_movies = user_item_matrix.loc[similar_users.index].mean(axis=0)
    user_movies = user_item_matrix.loc[user_id].dropna()
    recommendations = similar_users_movies.drop(user_movies.index).sort_values(ascending=False).head(top_n)
    return recommendations

In [5]:
column_names = ["user_id", "item_id", "rating", "timestamp"]
df = pd.read_csv("ml-100k/u.data", sep="\t", names=column_names)
print(df.head())

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [ ]:
user_item_matrix = df.pivot(index="user_id", columns="item_id", values="rating")
print(user_item_matrix.head(10))

item_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                              ...   
1         5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2         4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6         4.0   NaN   NaN   NaN   NaN   NaN   2.0   4.0   4.0   NaN  ...   
7         NaN   NaN   NaN   5.0   NaN   NaN   5.0   5.0   5.0   4.0  ...   
8         NaN   NaN   NaN   NaN   NaN   NaN   3.0   NaN   NaN   NaN  ...   
9         NaN   NaN   NaN   NaN   NaN   5.0   4.0   NaN   NaN   NaN  ...   
10        4.0   NaN   NaN   4.0   NaN   NaN   4.0   NaN   4.0   NaN  ...   

item_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id          

: 

In [ ]:
user_item_filled = user_item_matrix.fillna(0)
user_similarity = cosine_similarity(user_item_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print(user_similarity_df.head())

user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.166931  0.047460  0.064358  0.378475  0.430239  0.440367   
2        0.166931  1.000000  0.110591  0.178121  0.072979  0.245843  0.107328   
3        0.047460  0.110591  1.000000  0.344151  0.021245  0.072415  0.066137   
4        0.064358  0.178121  0.344151  1.000000  0.031804  0.068044  0.091230   
5        0.378475  0.072979  0.021245  0.031804  1.000000  0.237286  0.373600   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.319072  0.078138  0.376544  ...  0.369527  0.119482  0.274876   
2        0.103344  0.161048  0.159862  ...  0.156986  0.307942  0.358789   
3        0.083060  0.061040  0.065151  ...  0.031875  0.042753  0.163829   
4        0.188060  0.101284  0.060859  ...  0.052107

In [9]:
recommended_movies = recommend_movies(10, user_similarity_df, user_item_matrix, 5)
print(recommended_movies)
similar_users = get_similar_users(10, user_similarity_df, 5)
print(similar_users)

item_id
1463    5.0
543     5.0
262     5.0
644     5.0
853     5.0
dtype: float64
user_id
474    0.556142
6      0.551713
234    0.542308
308    0.538171
537    0.533171
Name: 10, dtype: float64
